In [31]:
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, CuDNNLSTM, Dense, Dropout,Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import string, os
import pandas as pd
import numpy as np
from keras import utils as np_utils
from tqdm import tqdm
import re

In [32]:
def get_lyrics(songs, artist, title=None):
    if title is None: return songs[(songs.artist == artist)]
    return songs[(songs.artist == artist) & (songs.title == title)]

In [33]:
songs = pd.read_csv('./datasets/labeled_lyrics_cleaned.csv',
                    usecols=["artist", "seq", "song"])
songs.rename(columns={"seq": "lyrics", "song": "title"}, inplace=True)
songs.drop_duplicates(inplace=True, ignore_index=True)

In [34]:
# Bob Marley, Bon Jovi, Boney M., Eminem, Iron Maiden, Madonna,
# R.E.M., Red Hot Chilli Peppers, The Beatles, The Rolling Stones, U2
# for name, his_songs in songs.groupby("artist"):
#     if len(his_songs) > 95:
#         print(name, len(his_songs))
# print('\n'.join(sorted(set(songs.artist))))
# del name, his_songs
print(next(iter(get_lyrics(songs, "Bob Marley", "Three Little Birds").lyrics)))
print(len(list(get_lyrics(songs, "Bob Marley").lyrics)))

Don't worry about a thing
'Cause every little thing gonna be alright
Singing' don't worry about a thing
'Cause every little thing gonna be alright

Rise up this mornin'
Smiled with the risin' sun
Three little birds
Pitch by my doorstep
Singin' sweet songs
Of melodies pure and true
Saying', (this is my message to you)

Singing' don't worry 'bout a thing
'Cause every little thing gonna be alright
Singing' don't worry (don't worry) 'bout a thing
'Cause every little thing gonna be alright

Rise up this mornin'
Smiled with the risin' sun
Three little birds
Pitch by my doorstep
Singin' sweet songs
Of melodies pure and true
Sayin', this is my message to you

Singin' don't worry about a thing, worry about a thing, oh
Every little thing gonna be alright, don't worry
Singin' don't worry about a thing, I won't worry
"'Cause every little thing gonna be alright

Singin' don't worry about a thing
'Cause every little thing gonna be alright, I won't worry
Singin', don't worry about a thing
'Cause ever

In [35]:

all_lyrics = list(set(get_lyrics(songs, "Bob Marley").lyrics))
lyrics_cnt = len(all_lyrics)
'''
all_lyrics = []
all_lyrics.extend(list(lyrics.values))
#print(all_lyrics)

for a in all_lyrics:
  if not isinstance(a, str):
    print("found one")
    all_lyrics.remove(a)

print(len(all_lyrics))
# all_lyrics = all_lyrics[:200]

translator = str.maketrans('', '', string.punctuation)

all_lyrics = all_lyrics[0:1000]
'''
None
# poetry = pd.read_csv('/content/gdrive/My Drive/Kaggle/PoetryFoundationData.csv',quotechar='"')

# poetry.head()

# poem = poetry.Poem
# poem_cnt = len(poem)

# all_poems = []
# all_poems.extend(list(poem.values))


# for a in all_poems:
#   if not isinstance(a, str):
#     all_poems.remove(a)

# all_poems = all_poems[:50]
# joined = all_poems+all_lyrics







'\nall_lyrics = []\nall_lyrics.extend(list(lyrics.values))\n#print(all_lyrics)\n\nfor a in all_lyrics:\n  if not isinstance(a, str):\n    print("found one")\n    all_lyrics.remove(a)\n\nprint(len(all_lyrics))\n# all_lyrics = all_lyrics[:200]\n\ntranslator = str.maketrans(\'\', \'\', string.punctuation)\n\nall_lyrics = all_lyrics[0:1000]\n'

In [150]:

def clean_text(txt):
    # txt = "".join(v for v in txt if v not in string.punctuation).lower()
    # txt = txt.encode("utf8").decode("ascii",'ignore')
    rgx = re.compile(r"\[[^\n\]]*]")
    return rgx.sub('', txt) 

def get_sequence_of_tokens(corpus,
                           lines_in_n_gram=2, 
                           endl_as_token=False,
                           ignoring_empty_lines=False):
    if endl_as_token:
        corpus.append(["endl"])
    tokenizer.fit_on_texts(corpus)
    corpus.pop()
    total_words = len(tokenizer.word_index) + 1

    input_sequences = []
    for song in tqdm(corpus):
        if ignoring_empty_lines:
            rgx = re.compile(r"(\r\n)+")
            song = rgx.sub("\r\n", song)
        if endl_as_token:
            song = song.replace("\r\n", " endl\r\n")
        print(song.split('\r\n'))
        token_lists = tokenizer.texts_to_sequences(song.split('\r\n'))
        print(token_lists)
        for i in range(lines_in_n_gram, len(token_lists)):
            flat = [jt for it in token_lists[i-lines_in_n_gram:i] for jt in it]
            for j in range(1, len(flat)):
                n_gram_sequence = flat[:j+1]
                input_sequences.append(n_gram_sequence)
        '''
        prev_token_list = []
        for line in song.split('\r\n'):
            if not line:
                continue
#             print(line)
            token_list = list(tokenizer.texts_to_sequences([line])[0])
            #print(token_list)
            # input_sequences.append(list(prev_token_list) + list(token_list))
            two_lines = prev_token_list + token_list
            for i in range(1, len(two_lines)):
                n_gram_sequence = two_lines[:i+1]
                input_sequences.append(n_gram_sequence)
#             print(input_sequences)
            prev_token_list = token_list
        '''
    return input_sequences, total_words

def get_padded_sequences(input_sequences):
    max_seq_len = max( [len(x) for x in input_sequences])
    padded_input_sequences =  np.array(pad_sequences(input_sequences, 
                                                     maxlen=max_seq_len, padding='pre'))
    return padded_input_sequences, max_seq_len


def prep_train_data(padded_input_sequences,total_words):
    
    x_train = padded_input_sequences[:,:-1]
    labels = padded_input_sequences[:,-1]
    labels = keras.utils.np_utils.to_categorical(labels, num_classes=total_words)
    
    return x_train, labels

In [151]:
# corpus = [clean_text(x) for x in all_lyrics]
corpus = all_lyrics
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r')
input_sequences, total_words = get_sequence_of_tokens(corpus, 4, True, False)
padded_input_sequences, max_seq_len = get_padded_sequences(input_sequences)
x_train, labels = prep_train_data(padded_input_sequences,total_words)

 15%|███████████▊                                                                    | 24/162 [00:00<00:01, 130.79it/s]

["Chances are we're gonna leave now endl", 'Sorry for the victim now endl', 'Although my days are filled with sorrow endl', 'I see here, a bright tomorrow endl', 'Chances, chances are some might not hold out endl', 'Chances are, hang on right now endl', 'Do do do do my days are filled with sorrow endl', 'Ill stay here til bright tomorrow endl', 'Chances, chances are some might not hold out endl', 'Chances are, hang on right now']
[[277, 52, 50, 34, 847, 18, 2372], [1087, 30, 2, 1088, 18, 2372], [1618, 15, 368, 52, 933, 76, 518, 2372], [3, 48, 213, 4, 848, 324, 2372], [277, 277, 52, 99, 435, 110, 252, 87, 2372], [277, 52, 489, 20, 38, 18, 2372], [26, 26, 26, 26, 15, 368, 52, 933, 76, 518, 2372], [1619, 934, 213, 1620, 848, 324, 2372], [277, 277, 52, 99, 435, 110, 252, 87, 2372], [277, 52, 489, 20, 38, 18]]
['Greetings in the name of his majesty, Emperor Haile Selassie I endl', 'Jah rastafari! endl', 'Ever-living, ever-feareful, ever-sure endl', 'Selassie I the first endl', 'Yeah endl', 

100%|███████████████████████████████████████████████████████████████████████████████| 162/162 [00:00<00:00, 586.41it/s]


["Roots Natty roots\nDread Binghi dread\nI and I are the roots\n\nSome are leaf\nSome are branches\nI and I are the roots\n\nSome are the dry wood\nFor the fire\nWhoa, Look at that\n\nThey need a dry wood\nTo cook their raw food\nWhoa, look at that\n\nGot to survive\nInthis man maniac downpression\nGot to survive\nIn iration\n\nThey say roots Natty roots\nDread Binghi dread\nI and I are the roots\n\nSome are wolf\nIn sheep's clothing\nWhoa, look at that\n\nMany are called\nA few are chosen\nWhoa, look at that\n\nNothing they can do\nTo separate I and I\nFrom the love of our fathers\n\nYou see, blood is thicker than water\nWhoa, look at that\n\nGot to survive\nIn the ghetto\nGot to survive\nPeople ....\n\nRoots Natty roots\nDread Binghi dread\nI and I are the roots\nRoots Natty\nDread Binghi\nI and I are the roots\n"]
[[211, 117, 211, 84, 641, 84, 3, 6, 3, 52, 2, 211, 99, 52, 1726, 99, 52, 1727, 3, 6, 3, 52, 2, 211, 99, 52, 2, 457, 871, 30, 2, 139, 280, 182, 156, 31, 44, 109, 4, 457, 87

In [152]:
print('\n'.join(sorted(tokenizer.word_index)), end="\n\n\n")
print('\n'.join(tokenizer.sequences_to_texts(input_sequences[:20])))
print(max_seq_len)

'alf
'and
'arder
'ave
'bout
'cause
'e
'em
'er
'ere
'i
'igh
'ighest
'n'
'nough
'pon
'round
'til
'till
'tis
'yp
'ypocrites
100
3
400
a
aaa
aaah
aah
aay
abandoned
abednego
abide
able
about
above
abroad
abundance
abused
accused
ace
achieve
across
act
actin'
acting
action
adventure
adventurer
advice
affection
afraid
afri
africa
african
africans
after
again
against
age
ages
aggression
ago
agree
ah
aim
aiming
ain't
air
alive
all
alligator
almighty
alone
along
already
alright
although
always
am
ambition
ambush
amen
america
an
an'
analyze
ancient
and
angels
angry
another
answer
any
anyhow
anyone
anything
anytime
anyway
anywhere
apart
are
aren't
arm
armageddon
armagiddyon
arms
around
arrival
as
ashamed
ashes
aside
ask
asking
astray
at
atomic
atrocity
attained
attraction
awaiting
awaits
awake
away
axe
ay
b
ba
bab
babe
babel
babies'
baby
babylon
back
backbiting
bad
badder
baf
bag
bake
baldheads
bank
bar
barnyard
bars
basic
basket
bastard
baton
batter
battle
be
beach
bear
beating
beats
because
beco

In [153]:
def lstm_model(max_seq_len,total_words):
    
    input_len = max_seq_len - 1 #zadnju predvidam
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(CuDNNLSTM(128))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = lstm_model(max_seq_len,total_words)
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 63, 10)            23730     
                                                                 
 cu_dnnlstm_6 (CuDNNLSTM)    (None, 128)               71680     
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 2373)              306117    
                                                                 
Total params: 401,527
Trainable params: 401,527
Non-trainable params: 0
_________________________________________________________________


In [154]:
model.fit(x_train, labels, epochs=20, verbose=1)

Epoch 1/20
4930/4930 [==============================] - 28s 6ms/step - loss: 5.2000
Epoch 2/20
4930/4930 [==============================] - 27s 6ms/step - loss: 4.1606
Epoch 3/20
4930/4930 [==============================] - 27s 5ms/step - loss: 3.3370
Epoch 4/20
4930/4930 [==============================] - 27s 5ms/step - loss: 2.7740
Epoch 5/20
4930/4930 [==============================] - 27s 6ms/step - loss: 2.3894
Epoch 6/20
4930/4930 [==============================] - 27s 6ms/step - loss: 2.1195
Epoch 7/20
4930/4930 [==============================] - 27s 6ms/step - loss: 1.9267
Epoch 8/20
4930/4930 [==============================] - 27s 5ms/step - loss: 1.7732
Epoch 9/20
4930/4930 [==============================] - 27s 5ms/step - loss: 1.6552
Epoch 10/20
4930/4930 [==============================] - 27s 6ms/step - loss: 1.5561
Epoch 11/20
4930/4930 [==============================] - 27s 5ms/step - loss: 1.4807
Epoch 12/20
4930/4930 [==============================] - 27s 5ms/step - lo

In [156]:
def generate_lyrics(seed_txt, next_words_cnt , max_seq_len, model):
  
    for i in range(0, next_words_cnt):
        token_list = tokenizer.texts_to_sequences([seed_txt])[0]
        padded_token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
        
        prediction = np.argmax(model.predict(padded_token_list), axis=-1)

        #print('prediction',prediction)
        #tokenizer.word_index.items() --> rijecnik (rijec,index)
        
        
        for (word, index) in tokenizer.word_index.items():
            output_word = ""
            if (prediction == index):
                seed_txt += " " + word
                break
      
        
    return seed_txt.title()

def generate_lyrics_endl_as_token(seed_txt,
                                  lines_in_n_gram,
                                  num_lines,
                                  max_line_len,
                                  max_seq_len,
                                  model):
        st_list = seed_txt.replace('\r', "").split('\n')
        st_list = st_list[max(0, len(st_list) - lines_in_n_gram):]
        token_list = tokenizer.texts_to_sequences(st_list)
        lines_remaining = num_lines - len(token_list)
        flat = [jt for it in token_list for jt in it]
        endl_token = tokenizer.texts_to_sequences(["endl"])[0][0]
        tokenized_song = []
        curr_line = []
        while lines_remaining:
            # print(lines_remaining)
            if len(curr_line) >= max_line_len:
                prediction = endl_token
            else:
                padded_token_list = pad_sequences([flat], maxlen=max_seq_len-1, padding='pre')
                prediction = np.argmax(model.predict(padded_token_list), axis=-1)[0]
            curr_line.append(prediction)
            flat.append(prediction)
            if prediction == endl_token:
                lines_remaining -= 1
                if len(token_list) >= lines_in_n_gram: del token_list[0]
                token_list.append(curr_line)
                tokenized_song.append(curr_line[:-1])
                curr_line = []
                flat = [jt for it in token_list for jt in it]
        # print(tokenized_song[1:])
        print('\n'.join(
            tokenizer.sequences_to_texts(tokenized_song)
        ))

In [157]:
# model.save("model_200")

In [158]:
import random
from collections import defaultdict
unique = []
for i in range(0,len(corpus)):
    unique.append(corpus[i].split())

'''
for i in range(0,20):
  random_list = unique[random.randint(0,len(unique)-1)]
  seed = random_list[random.randint(0,len(random_list))]
  print( generate_lyrics(seed,random.randint(3,15),max_seq_len, model ) )
'''
# duljina pjesme iz iste distribucije kao ostale
# koristenje corpusa tu?
num_lines = random.choice([it.count("\r\n") for it in all_lyrics])
max_line_len = max([max([len(it.split()) for it in song.split("\r\n")]) for song in all_lyrics])


seed_txt = """Here's a little song I wrote
You might want to sing it note for note
Don't worry, be happy"""

generate_lyrics_endl_as_token(seed_txt,
                              lines_in_n_gram=4,
                              num_lines=num_lines,
                              max_line_len=max_line_len,
                              max_seq_len=max_seq_len,
                              model=model)


brother
i want to love you through
we don't need no trouble
what we need is love
what we need is love love
love i know now
lord i thank you

i said i heard my mother
so every little thing i ever had

when you wake up early in the morning
and then the children the truth

we're jammin' we're jammin' we're jammin' jammin'
we're jammin' we're jammin' jammin'
we're jammin' we're jammin' jammin'
we're jammin' we're jammin' jammin'
we're jammin' we're jammin' jammin'
we're jammin' we're jammin' we're jammin' we're jammin'

we're jammin' we're jammin'

i want to jam it wid you
we're jammin' jammin' jammin' jammin'

i want to jam it wid you
we're jammin' jammin' jammin' jammin'

i want to jam it wid you
we're jammin' jammin' jammin' jammin'

i want to jam it wid you
we're jammin' jammin' jammin' jammin'

i want to jam it wid you
